In [1]:
import numpy as np

import transformers

import tensorflow as tf
import tensorflow_datasets as tfds

import networkx as nx
from pyvis.network import Network

/home/insane/.cache/pypoetry/virtualenvs/channelexplorer-ajWhp0a7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-16 21:40:30.053235: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 21:40:30.053259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 21:40:30.054153: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-16 21:40:30.058668: I tensorflow/core/platform/cpu

In [18]:
def pretty_print(obj, indent=0):
    spacing = ' ' * indent
    if isinstance(obj, dict):
        print(f"{spacing}{{")
        for key, value in obj.items():
            print(f"{spacing}  {key}:")
            pretty_print(value, indent + 4)
        print(f"{spacing}}}")
    elif isinstance(obj, (list, tuple)):
        print(f"{spacing}[")
        for item in obj:
            pretty_print(item, indent + 4)
        print(f"{spacing}]")
    elif isinstance(obj, np.ndarray):
        print(f"{spacing}np.array[{obj.dtype}] (shape: {obj.shape})")
    elif isinstance(obj, tf.Tensor):
        print(f"{spacing}tf.Tensor[{obj.dtype}] (shape: {obj.shape})")
    elif isinstance(obj, tf.keras.layers.Layer):
        print(f"{spacing}tf.keras.layers.Layer[{obj.dtype}] (shape: {obj.shape})")
    elif isinstance(obj, transformers.tokenization_utils_base.BatchEncoding):
        print(f"{spacing}BatchEncoding (keys: {list(obj.keys())})")
        for key, value in obj.items():
            print(f"{spacing}  {key}:")
            pretty_print(value, indent + 4)
    else:
        print(f"{spacing}{obj}")

In [3]:
vgg16 = tf.keras.applications.VGG16(weights='imagenet')
inceptionv3 = tf.keras.applications.InceptionV3(weights='imagenet')
gpt2 = transformers.TFAutoModelForCausalLM.from_pretrained('gpt2')
gpt2.config.pad_token_id = gpt2.config.eos_token_id
vit = transformers.TFViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

imagenette = tfds.load('imagenette/160px', split='train', as_supervised=False, batch_size=None)
gpt2_custom = [
    'Hello world, sdjfklasdj ksdj fskdj lfskdj fklsadjf lskadjf laskdjf klasdjf klasd jfklasdj fklasd jfklasd jflaksjdf klasdjf aklsdjf klasjd fklasdj fklasfj',
    'I love you',
    'This is my world',
    'Hello, how are you? I am fine, thank you!',
    'Tell me a story on the moon',
]

2024-09-16 21:40:31.566578: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-16 21:40:31.587342: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel f

In [33]:
def vgg16_preprocess(x):
    x = tf.image.resize(x, [224, 224])
    x = tf.keras.applications.vgg16.preprocess_input(x)
    return x

def inceptionv3_preprocess(x):
    x = tf.image.resize(x, [299, 299])
    x = tf.keras.applications.inception_v3.preprocess_input(x)
    return x

def gpt2_preprocess(x):
    tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2')
    x = tokenizer(x, return_tensors='tf')
    return x

def vit_preprocessor(x):
    feature_extractor = transformers.ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
    x = x['image']
    x = tf.image.resize(x, (224, 224)) / 255.0
    x = feature_extractor(images=x, return_tensors="tf")
    x = x['pixel_values']
    return x

input_vgg16 = np.array([vgg16_preprocess(x['image']) for x in imagenette.take(5)])
input_inceptionv3 = np.array([inceptionv3_preprocess(x['image']) for x in imagenette.take(5)])
input_gpt2 = [gpt2_preprocess(x) for x in gpt2_custom]
input_vit = np.concatenate([vit_preprocessor(x) for x in imagenette.take(5)], axis=0)

pretty_print(input_vgg16)
pretty_print(input_inceptionv3)
pretty_print(input_gpt2)
pretty_print(input_vit)

2024-09-16 22:11:34.608081: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-09-16 22:11:34.638044: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


np.array[float32] (shape: (5, 224, 224, 3))
np.array[float32] (shape: (5, 299, 299, 3))
[
    BatchEncoding (keys: ['input_ids', 'attention_mask'])
      input_ids:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 79))
      attention_mask:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 79))
    BatchEncoding (keys: ['input_ids', 'attention_mask'])
      input_ids:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 3))
      attention_mask:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 3))
    BatchEncoding (keys: ['input_ids', 'attention_mask'])
      input_ids:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 4))
      attention_mask:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 4))
    BatchEncoding (keys: ['input_ids', 'attention_mask'])
      input_ids:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 13))
      attention_mask:
        tf.Tensor[<dtype: 'int32'>] (shape: (1, 13))
    BatchEncoding (keys: ['input_ids', 'attention_mask'])
      input_ids:
        tf.Tensor[<dtype: '

2024-09-16 22:11:36.004227: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [34]:
vgg16_layers = list(
    map(
        lambda l: l.name,
        vgg16.layers
    )
)

inceptionv3_layers = list(
    map(
        lambda l: l.name,
        inceptionv3.layers
    )
)

gpt2_layers = list(
    map(
        lambda l: l.name,
        gpt2.layers
    )
)

vit_layers = list(
    map(
        lambda l: l.name,
        vit.layers
    )
)

print(vgg16_layers)
print(inceptionv3_layers)
print(gpt2_layers)
print(vit_layers)

['input_1', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool', 'flatten', 'fc1', 'fc2', 'predictions']
['input_2', 'conv2d', 'batch_normalization', 'activation', 'conv2d_1', 'batch_normalization_1', 'activation_1', 'conv2d_2', 'batch_normalization_2', 'activation_2', 'max_pooling2d', 'conv2d_3', 'batch_normalization_3', 'activation_3', 'conv2d_4', 'batch_normalization_4', 'activation_4', 'max_pooling2d_1', 'conv2d_8', 'batch_normalization_8', 'activation_8', 'conv2d_6', 'conv2d_9', 'batch_normalization_6', 'batch_normalization_9', 'activation_6', 'activation_9', 'average_pooling2d', 'conv2d_5', 'conv2d_7', 'conv2d_10', 'conv2d_11', 'batch_normalization_5', 'batch_normalization_7', 'batch_normalization_10', 'batch_normalization_11', 'activation_5', 'activatio

In [35]:
# Function to extract activations from self-attention, feed-forward, and hidden states
def get_transformer_block_activations(model, inputs):
    """
    Function to get intermediate activations from each transformer block, including self-attention and feed-forward activations.
    
    Args:
    - model: Transformer model from Hugging Face (e.g., GPT-2)
    - inputs: Tokenized input (output of tokenizer)
    
    Returns:
    - Dictionary containing activations for self-attention, feed-forward, and hidden states
    """
    activations = {}
    for i in inputs:
        # Run the model with output_hidden_states and output_attentions to get all intermediate data
        outputs = model(input_ids=i['input_ids'], output_hidden_states=True, output_attentions=True)

        # The hidden_states contains the hidden layer outputs, and attentions contains attention scores
        hidden_states = outputs.hidden_states
        attentions = outputs.attentions

        activations_block = {
            "embedding_layer": hidden_states[0].numpy(),  # First hidden state is the embedding layer
        }

        # Iterate over the transformer blocks
        for i, (attention, hidden_state) in enumerate(zip(attentions, hidden_states[1:])):
            # Self-attention output (stored in attentions)
            activations_block[f"block_{i}_self_attention"] = attention.numpy()
            
            # Feed-forward output (this is typically captured in the hidden states after attention and feed-forward pass)
            activations_block[f"block_{i}_feed_forward"] = hidden_state.numpy()
        
        for k, v in activations_block.items():
            if k not in activations:
                activations[k] = []
            activations[k].append(v)
            
    return activations


In [42]:
def get_vit_block_activations(model, inputs):
    """
    Function to get intermediate activations from each transformer block in a ViT model, including self-attention and feed-forward activations.
    
    Args:
    - model: Vision Transformer model from Hugging Face.
    - inputs: Preprocessed image inputs (output of feature extractor).
    
    Returns:
    - Dictionary containing activations for self-attention, feed-forward, and hidden states from each block.
    """
    # Run the model with output_hidden_states and output_attentions to get all intermediate activations
    outputs = model(inputs, output_hidden_states=True, output_attentions=True)
    
    # Hidden states contain outputs from all layers; attentions contain attention maps
    logits = outputs.logits
    hidden_states = outputs.hidden_states
    attentions = outputs.attentions
    
    activations = {
        "embedding_layer": hidden_states[0],  # First hidden state is the embedding layer
    }

    # Iterate over the transformer blocks in the ViT model
    for i, (attention, hidden_state) in enumerate(zip(attentions, hidden_states[1:])):
        # Self-attention output (stored in attentions)
        activations[f"block_{i}_self_attention"] = attention
        
        # Feed-forward output (captured in the hidden states)
        activations[f"block_{i}_feed_forward"] = hidden_state

    return activations


In [67]:
def extract_activations_graph(model, inputs):
    G = nx.DiGraph()
    last_output_tensor = inputs
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.InputLayer, )):
            G.add_node(
                layer.name,
                name=layer.name,
                layer_type=layer.__class__.__name__,
                tensor_type=str(layer.dtype),
                input_shape=layer.input_shape,
                output_shape=layer.output_shape,
                output_tensor=inputs,
                is_parent=False,
                parent=None,
            )
            


        elif isinstance(layer, (transformers.TFGPT2MainLayer, transformers.TFGPT2LMHeadModel)):
            block_activations = get_transformer_block_activations(model, inputs)
            G.add_node(
                layer.name,
                name=layer.name,
                layer_type="TransformerBlock",
                config=layer.config.to_dict(),
                last_hidden_state_shape=tuple(layer.compute_output_shape(tf.TensorShape([None, model.config.n_positions]))['last_hidden_state']),
                past_key_values_shape=tuple(layer.compute_output_shape(tf.TensorShape([None, model.config.n_positions]))['past_key_values']),
                tensor_type=str(layer.dtype),
                parent=None,
                is_parent=True,
            )
            prev_block_name = None
            layer_type_map = {
                "embedding": "Embedding",
                "self_attention": "SelfAttention",
                "feed_forward": "FeedForward",
            }
            def get_layer_type(layer_name):
                for key, value in layer_type_map.items():
                    if key in layer_name:
                        return value
                return "Unknown"
            for layer_name, activation in block_activations.items():
                G.add_node(
                    layer_name,
                    name=layer_name,
                    layer_type=get_layer_type(layer_name),
                    output_shape=f"({activation[0].shape[0]}, InputLength, {activation[0].shape[2]})",
                    tensor_type=str(activation[0].dtype),
                    output_tensor=activation,
                    parent=layer.name,
                    is_parent=False,
                )
                if prev_block_name:
                    G.add_edge(prev_block_name, layer_name)
                prev_block_name = layer_name
                last_output_tensor = activation
                
                
        elif isinstance(layer, transformers.models.vit.modeling_tf_vit.TFViTMainLayer):
            block_activations = get_vit_block_activations(model, inputs)
            G.add_node(
                layer.name,
                name=layer.name,
                layer_type="TransformerBlock",
                config=layer.config.to_dict(),
                tensor_type=str(layer.dtype),
                parent=None,
                is_parent=True,
            )
            prev_block_name = None
            layer_type_map = {
                "embedding": "Embedding",
                "self_attention": "SelfAttention",
                "feed_forward": "FeedForward",
            }
            def get_layer_type(layer_name):
                for key, value in layer_type_map.items():
                    if key in layer_name:
                        return value
                return "Unknown"
            for layer_name, activation in block_activations.items():
                G.add_node(
                    layer_name,
                    name=layer_name,
                    layer_type=get_layer_type(layer_name),
                    output_shape=[a.shape for a in activation],
                    tensor_type=str(activation[0].dtype),
                    output_tensor=activation,
                    parent=layer.name,
                    is_parent=False,
                )
                if prev_block_name:
                    G.add_edge(prev_block_name, layer_name)
                prev_block_name = layer_name
                last_output_tensor = activation
        elif isinstance(
            layer,
            (
                tf.keras.layers.Conv2D,
                tf.keras.layers.Concatenate,
                tf.keras.layers.MaxPooling2D,
                tf.keras.layers.BatchNormalization,
                tf.keras.layers.Activation,
                tf.keras.layers.AveragePooling2D,
                tf.keras.layers.GlobalAveragePooling2D,
                tf.keras.layers.GlobalMaxPooling2D,
            ),
        ):
            intermediate_model = tf.keras.Model(inputs=model.input, outputs=layer.output)
            intermediate_output = intermediate_model(inputs)
            G.add_node(
                layer.name,
                name=layer.name,
                input_shape=layer.input_shape,
                layer_type=layer.__class__.__name__,
                output_shape=layer.output_shape,
                output_tensor=intermediate_output.numpy(),
                tensor_type=layer.dtype,
                is_parent=False,
                parent=None,
            )
            for inbound_node in layer._inbound_nodes:
                if isinstance(inbound_node.inbound_layers, list):
                    for inbound_layer in inbound_node.inbound_layers:
                        G.add_edge(inbound_layer.name, layer.name)
                else:
                    G.add_edge(inbound_node.inbound_layers.name, layer.name)
            last_output_tensor = intermediate_output
                    
        elif isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Flatten)):
            intermediate_output = layer(last_output_tensor)
            G.add_node(
                layer.name,
                name=layer.name,
                layer_type=layer.__class__.__name__,
                input_shape=last_output_tensor.shape,
                output_shape=intermediate_output.shape,
                output_tensor=intermediate_output.numpy(),
                tensor_type=layer.dtype,
                is_parent=False,
                parent=None,
            )
            for inbound_node in layer._inbound_nodes:
                if isinstance(inbound_node.inbound_layers, list):
                    for inbound_layer in inbound_node.inbound_layers:
                        G.add_edge(inbound_layer.name, layer.name)
                else:
                    G.add_edge(inbound_node.inbound_layers.name, layer.name)
            last_output_tensor = intermediate_output
        else:
            print("Is a generic layer", layer.name, layer)
            raise Exception("Is a generic layer")
    return G

# model_graph = extract_activations_graph(vgg16, input_vgg16)
# pretty_print(list(model_graph.nodes.data()))
# model_graph = extract_activations_graph(inceptionv3, input_inceptionv3)
# pretty_print(list(model_graph.nodes.data()))
# model_graph = extract_activations_graph(gpt2, input_gpt2)
# pretty_print(list(model_graph.nodes.data()))
model_graph = extract_activations_graph(vit, input_vit)
pretty_print(list(model_graph.nodes.data()))

[
    [
        vit
        {
          name:
            vit
          layer_type:
            TransformerBlock
          config:
            {
              return_dict:
                True
              output_hidden_states:
                False
              output_attentions:
                False
              torchscript:
                False
              torch_dtype:
                None
              use_bfloat16:
                False
              tf_legacy_loss:
                False
              pruned_heads:
                {
                }
              tie_word_embeddings:
                True
              chunk_size_feed_forward:
                0
              is_encoder_decoder:
                False
              is_decoder:
                False
              cross_attention_hidden_size:
                None
              add_cross_attention:
                False
              tie_encoder_decoder:
                False
              max_length:
          